In [103]:
import pandas as pd
import requests
import json
from pandas import json_normalize
import logging

logging.basicConfig(filename='data_retrieval.log', level=logging.INFO)


In [52]:
sid = '09dceb980747489297e7ea0a51077a66'
skey = 'f3543385c18c48cd98018f4cc39bf12a'
baseURL = 'https://api.spotify.com'


### Endpoints

In [91]:
def request_access_token(cid = sid, skey = skey ):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": cid,
        "client_secret": skey
    }

    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json().get('access_token')
    else:
        print(f"Failed to obtain token, status code: {response.status_code}")
        return None

def fetch_tracks_info(track_ids):
    ids = ','.join(track_ids)
    url = f"https://api.spotify.com/v1/tracks?ids={ids}"

    response = requests.get(url, headers=headers)
    
    return response


### Endpoint utils

In [54]:
def new_session_headers():
    headers = {
        "Authorization": f'Bearer {request_access_token()}',
    }

    return headers

In [55]:
headers = new_session_headers()

### Core data

In [15]:
all_files = [f"./data_2022/MyData_streaming_hist/endsong_{i}.json" for i in range(12)]
df_list = [pd.read_json(file) for file in all_files]
df = pd.concat(df_list, ignore_index=True)
df.ts = pd.to_datetime(df.ts)
df = df.sort_values(by='ts').reset_index(drop = True)

df = df[['spotify_track_uri', 'ts']].copy()

### Metadata retrieval

In [84]:
def get_batch_metadata(batch_list):
    fetched_info = fetch_tracks_info(batch_list)
    if fetched_info.status_code == 200:
        fetched_info = json.loads(fetched_info.text)
        frames = [json_normalize(track) for track in fetched_info['tracks']]
        df = pd.concat(frames, ignore_index=True)

        return df
    else:
        logging

In [98]:
batch_list = ['7crplFzt7spG80NCbpmuCp', '1tYt8PbpbeTuqsNmprAZYY']

t = fetch_tracks_info(batch_list)

In [102]:
json.loads(t.text)['tracks']

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7n2wHs1TKAczGzO7Dd2rGr'},
     'href': 'https://api.spotify.com/v1/artists/7n2wHs1TKAczGzO7Dd2rGr',
     'id': '7n2wHs1TKAczGzO7Dd2rGr',
     'name': 'Shawn Mendes',
     'type': 'artist',
     'uri': 'spotify:artist:7n2wHs1TKAczGzO7Dd2rGr'}],
   'available_markets': [],
   'external_urls': {'spotify': 'https://open.spotify.com/album/2Ri94NAjmRjZkOgqoQCK8R'},
   'href': 'https://api.spotify.com/v1/albums/2Ri94NAjmRjZkOgqoQCK8R',
   'id': '2Ri94NAjmRjZkOgqoQCK8R',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b27360838ee79f90f9f038391d81',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e0260838ee79f90f9f038391d81',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d0000485160838ee79f90f9f038391d81',
     'width': 64}],
   'name': 'Handwritten (Revisited)',
   'release_

In [96]:
t

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,...,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,[],1,223826,False,https://api.spotify.com/v1/tracks/7crplFzt7spG...,7crplFzt7spG80NCbpmuCp,False,I Know What You Did Last Summer,0,...,2Ri94NAjmRjZkOgqoQCK8R,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Handwritten (Revisited),2015-11-20,day,16,album,spotify:album:2Ri94NAjmRjZkOgqoQCK8R,USUM71516597,https://open.spotify.com/track/7crplFzt7spG80N...
1,[{'external_urls': {'spotify': 'https://open.s...,[],1,210983,False,https://api.spotify.com/v1/tracks/1tYt8PbpbeTu...,1tYt8PbpbeTuqsNmprAZYY,False,Dessert,0,...,4xNCtUWlbfoRilthUHkWR1,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Dessert,2015-03-13,day,1,album,spotify:album:4xNCtUWlbfoRilthUHkWR1,USUM71502647,https://open.spotify.com/track/1tYt8PbpbeTuqsN...
